# Train a new model

In this notebook we will learn how to train a new model for axon & myelin segmentation. It covers the following scenario:

* Train a model from scratch by defining the parameters of the network
* Make inference using the trained model

**Important:** If you have access to a GPU card, we strongly recommend you use it. To do so, TensorFlow needs to be uninstalled and then re-installed using the GPU mode:
```bash
source activate ads_venv  # Activate virtual environment. This line might depend on your ADS installation
pip uninstall tensorflow
pip install tensorflow-gpu==1.13.1
```

In [1]:
import os
import json
import time
from pathlib import Path
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util import Retry

# Scientific package imports
import imageio
import numpy as np
import tensorflow as tf
from skimage import io
import matplotlib.pyplot as plt

# Utils import
from shutil import copy
import zipfile
from tqdm import tqdm
import cgi
import tempfile

# AxonDeepSeg imports
try:
    from AxonDeepSeg.ads_utils import download_data
except ModuleNotFoundError:
    # Change cwd to project main folder 
    os.chdir("..")
    try :
        from AxonDeepSeg.ads_utils import download_data
    except:
        raise
except:
    raise
# If no exceptions were raised import all folders        
from AxonDeepSeg.config_tools import validate_config
from AxonDeepSeg.train_network import train_model
from AxonDeepSeg.apply_model import axon_segmentation
import AxonDeepSeg.ads_utils as ads

# reset the tensorflow graph for new training
tf.reset_default_graph()

%matplotlib inline

Using TensorFlow backend.


### 0. Download example data

Before running this notebook, please make sure you have split, patched, and organized your data in the correct way by running the [guide_dataset_building.ipynb](guide_dataset_building.ipynb) tutorial.

### 1. Train a new model
#### 1.1. Define the name and path of the training set

Here we assume that the training data folder has already been created by following the guidelines detailed in [guide_dataset_building.ipynb](https://github.com/neuropoly/axondeepseg/blob/master/notebooks/guide_dataset_building.ipynb).

The expected structure of the training data folder is the following:

~~~
data
 └── Train
      └── image_0.png
      └── mask_0.png
      └── image_1.png
      └── mask_1.png
          ...
 └── Validation
      └── image_0.png
      └── mask_0.png
      └── image_1.png
      └── mask_1.png
          ...
~~~

In [5]:
path_training = Path("./training")  #  folder containing training data

#### 1.2. Define the U-Net architecture and hyper-parameters

Here we defined the network and training parameters (i.e. hyperparameters). We use a lighter network than the ones used in the original [AxonDeepSeg article](https://www.nature.com/articles/s41598-018-22181-4), because they require large GPU memory (>12GB). The network below runs on an NVIDIA TitanX in ~2h. Note that the architecture below might not produce satisfactory results on your data so you likely need to play around with the architecture and hyperparameters.

**Important:** The pixel size is not defined at the training step. During inference however, the parameter `-t {SEM,TEM}` sets the resampling resolution to 0.1µm or 0.01µm respectively (i.e., implying the pixel size of the training data should be around 0.1µm for SEM and 0.01µm for TEM). This is definitely a limitation of the current version of AxonDeepSeg, which we are planning to solve at some point (for more info, see [Issue #152](https://github.com/neuropoly/axondeepseg/issues/152)). 

**Note about data augmentation:**
For each type of data augmentation, the order needs to be specified if you decide to apply more than one transformation sequentially. For instance, setting `da-0-shifting-activate` to `True` means that the shifting is the first transformation that will be applied to the sample(s) during training. The default ranges of transformations are:
- **Shifing**: Random horizontal and vertical shifting between 0 and 10% of the patch size, sampled from a uniform distribution.
- **Rotation**: Random rotation, angle between 5 and 89 degrees, sampled from a uniform distribution.
- **Rescaling**: Random rescaling of a randomly sampled factor between 1/1.2 and 1.2.
- **Flipping**: Random fipping: vertical fipping or horizontal fipping.
- **Blurring**: Gaussian blur with the standard deviation of the gaussian kernel being uniformly sampled between 0 and 4.
- **Elastic deformation**: Random elastic deformation with uniformly sampled deformation coefficient α=[1–8] and fixed standard deviation σ=4.

You can find more information about the range of transformations applied to the patches for each data augmentation technique in the file [data_augmentation.py](https://github.com/neuropoly/axondeepseg/blob/master/AxonDeepSeg/data_management/data_augmentation.py).

In [2]:
# Example of network configuration for TEM data (small network trainable on a Titan X GPU card)
config = {
    
# General parameters:    
  "n_classes": 3,  # Number of classes. For this application, the number of classes should be set to **3** (i.e. axon pixel, myelin pixel, or background pixel).
  "thresholds": [0, 0.2, 0.8],  # Thresholds for the 3-class classification problem. Do not modify.  
  "trainingset_patchsize": 512,  # Patch size of the training set in pixels (note that the patches have the same size in both dimensions).  
  "trainingset": "TEM_3c_512",  # Name of the training set.
  "batch_size": 8,  # Batch size, i.e. the number of training patches used in one iteration of the training. Note that a larger batch size will take more memory.
  "epochs":600,
  "checkpoint_period": 5, # Number of epoch after which the model checkpoint is saved.
  "checkpoint": None, # Checkpoint to use to resume training. Option: "loss", "accuracy" or None.

# Network architecture parameters:     
  "depth": 4,  # Depth of the network (i.e. number of blocks of the U-net).
  "convolution_per_layer": [2, 2, 2, 2],  # Number of convolution layers used at each block.
  "size_of_convolutions_per_layer": [[5, 5], [3, 3], [3, 3], [3, 3]],  # Kernel size of each convolution layer of the network.
  "features_per_convolution": [[[1, 16], [16, 16]], [[16, 32], [32, 32]], [[32, 64], [64, 64]], [[64, 128], [128, 128]]],  # Number of features of each convolution layer.
  "downsampling": "convolution",  # Type of downsampling to use in the downsampling layers of the network. Option "maxpooling" for standard max pooling layer or option "convolution" for learned convolutional downsampling.
  "dropout": 0.75,  # Dropout to use for the training. Note: In TensorFlow, the keep probability is used instead. For instance, setting this param. to 0.75 means that 75% of the neurons of the network will be kept (i.e. dropout of 25%).
     
# Learning rate parameters:    
  "learning_rate": 0.01,  # Learning rate to use in the training.  
  "learning_rate_decay_activate": True,  # Set to "True" to use a decay on the learning rate.  
  "learning_rate_decay_period": 24000,  # Period of the learning rate decay, expressed in number of images (samples) seen.
  "learning_rate_decay_type": "polynomial",  # Type of decay to use. An exponential decay will be used by default unless this param. is set to "polynomial" (to use a polynomial decay).
  "learning_rate_decay_rate": 0.99,  # Rate of the decay to use for the exponential decay. This only applies when the user does not set the decay type to "polynomial".
    
# Batch normalization parameters:     
  "batch_norm_activate": True,  # Set to "True" to use batch normalization during the training.
  "batch_norm_decay_decay_activate": True,  # Set to "True" to activate an exponential decay for the batch normalization step of the training.  
  "batch_norm_decay_starting_decay": 0.7,  # The starting decay value for the batch normalization. 
  "batch_norm_decay_ending_decay": 0.9,  # The ending decay value for the batch normalization.
  "batch_norm_decay_decay_period": 16000,  # Period of the batch normalization decay, expressed in number of images (samples) seen.
        
# Weighted cost parameters:    
  "weighted_cost-activate": True,  # Set to "True" to use weights based on the class in the cost function for the training.
  "weighted_cost-balanced_activate": True,  # Set to "True" to use weights in the cost function to correct class imbalance. 
  "weighted_cost-balanced_weights": [1.1, 1, 1.3],  # Values of the weights for the class imbalance. Typically, larger weights are assigned to classes with less pixels to add more penalty in the cost function when there is a misclassification. Order of the classes in the weights list: background, myelin, axon.
  "weighted_cost-boundaries_sigma": 2,  # Set to "True" to add weights to the boundaries (e.g. penalize more when misclassification happens in the axon-myelin interface).
  "weighted_cost-boundaries_activate": False,  # Value to control the distribution of the boundary weights (if activated). 
    
# Data augmentation parameters:
  "da-type": "all",  # Type of data augmentation procedure. Option "all" applies all selected data augmentation transformations sequentially, while option "random" only applies one of the selected transformations (randomly) to the sample(s). List of available data augmentation transformations: 'random_rotation', 'noise_addition', 'elastic', 'shifting', 'rescaling' and 'flipping'. 
  "da-0-shifting-activate": True, 
  "da-1-rescaling-activate": False,
  "da-2-random_rotation-activate": False,  
  "da-3-elastic-activate": True, 
  "da-4-flipping-activate": True, 
  "da-5-noise_addition-activate": False,
  "da-6-reflection_border-activate": False
}


In [3]:
config = {
  "n_classes": 3,
  "thresholds": [
    0,
    0.2,
    0.8
  ],
  "trainingset_patchsize": 1250,
  "trainingset": "TEM_3c_512",
  "batch_size": 8,
  "epochs": 2000,
  "depth": 4,
  "convolution_per_layer": [
    2,
    2,
    2,
    2
  ],
  "size_of_convolutions_per_layer": [
    [
      5,
      5
    ],
    [
      3,
      3
    ],
    [
      3,
      3
    ],
    [
      3,
      3
    ]
  ],
  "features_per_convolution": [
    [
      [
        1,
        16
      ],
      [
        16,
        16
      ]
    ],
    [
      [
        16,
        32
      ],
      [
        32,
        32
      ]
    ],
    [
      [
        32,
        64
      ],
      [
        64,
        64
      ]
    ],
    [
      [
        64,
        128
      ],
      [
        128,
        128
      ]
    ]
  ],
  "downsampling": "convolution",
  "dropout": 0.75,
  "learning_rate": 0.01,
  "learning_rate_decay_activate": True,
  "learning_rate_decay_period": 24000,
  "learning_rate_decay_type": "polynomial",
  "learning_rate_decay_rate": 0.99,
  "batch_norm_activate": True,
  "batch_norm_decay_decay_activate": True,
  "batch_norm_decay_starting_decay": 0.7,
  "batch_norm_decay_ending_decay": 0.9,
  "batch_norm_decay_decay_period": 16000,
  "weighted_cost-activate": True,
  "weighted_cost-balanced_activate": True,
  "weighted_cost-balanced_weights": [
    1.1,
    1,
    1.3
  ],
  "weighted_cost-boundaries_sigma": 2,
  "weighted_cost-boundaries_activate": False,
  "da-type": "all",
  "da-0-shifting-activate": True,
  "da-1-rescaling-activate": False,
  "da-2-random_rotation-activate": False,
  "da-3-elastic-activate": True,
  "da-4-flipping-activate": True,
  "da-5-noise_addition-activate": False,
  "da-6-reflection_border-activate": False
}

#### 1.3. Define training path and save configuration parameters

Here we define the path where the new model will be saved. It is useful to add date+time in path definition in case multiple training are launched (to avoid conflicts).

The network configuration parameters defined at 1.2. are saved into a .json file in the model folder. This .json file keeps tract of the network and model parameters in a structured way.

In [7]:
# Define path to where the trained model will be saved
dir_name = Path(config["trainingset"] + '_' + time.strftime("%Y-%m-%d") + '_' + time.strftime("%H-%M-%S"))
path_model = "../models" / dir_name

print("This training session's model will be saved in the folder: " + str(path_model.resolve().absolute()))

# Create directory
if not os.path.exists(path_model):
    os.makedirs(path_model)

# Define file name of network configuration
file_config = 'config_network.json'

# Load/Write config file (depending if it already exists or not)
fname_config = os.path.join(path_model, file_config)
if os.path.exists(fname_config):
    with open(fname_config, 'r') as fd:
        config_network = json.loads(fd.read())
else:
    with open(fname_config, 'w') as f:
        json.dump(config, f, indent=2)
    with open(fname_config, 'r') as fd:
        config_network = json.loads(fd.read())

This training session's model will be saved in the folder: /home/ubuntu/code/libs/axondeepseg/models/TEM_3c_512_2020-08-18_08-27-26


#### 1.4. Launch the training procedure

The training can be launched by calling the *'train_model'* function. After each epoch, the function will display the loss and accuracy of the model. The model checkpoints will be saved according to the "checkpoint_period" parameter in "config".

In [8]:
# reset the tensorflow graph for new testing
tf.reset_default_graph()

train_model(str(path_training), str(path_model), config)
# Note: For multi-OS compatibility of this notebook, paths were defined as Path objects from the pathlib module.
# They need to be converted into strings prior to be given as arguments to train_model(), as some old-style string
# concatenation (e.g. "+") are still used in it.
# In your own application, simply defining paths with correct syntax for your OS as strings instead of Path
# objects would be sufficient.












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.







Epoch 1/600
11/11 [==============================] - 132s 12s/step - loss: 0.4680 - acc: 0.6681 - dice_axon: 0.0835 - dice_myelin: 0.2850 - val_loss: 0.4217 - val_acc: 0.7026 - val_dice_axon: 0.0985 - val_dice_myelin: 0.2939

Epoch 2/600
11/11 [==============================] - 128s 12s/step - loss: 0.2621 - acc: 0.7818 - dice_axon: 0.0707 - dice_myelin: 0.3630 - val_loss: 0.2375 - val_acc: 0.7877 - val_dice_axon: 0.0684 - val_dice_myelin: 0.4668
Epoch 3/600
11/11 [==============================] - 128s 12s/step - loss: 0.2153 - acc: 0.7710 - dice_axon: 0.0557 - dice_myelin: 0.3563 - val_loss: 0.1338 - val_acc: 0.8811 - val_dice_axon: 0.0392 - val_dice_myelin: 0.5758
Epoch 4/600
11/11 [==============================] - 127s 12s/step - loss: 0.1821 - acc: 0.7938 - dice_axon: 0.0406 - dice_myelin: 0.4117 - val_loss: 0.2052 - val_acc: 0.8017 - val_dice_axon:

11/11 [==============================] - 119s 11s/step - loss: 0.1385 - acc: 0.8322 - dice_axon: 0.0050 - dice_myelin: 0.5254 - val_loss: 0.1948 - val_acc: 0.8054 - val_dice_axon: 0.0011 - val_dice_myelin: 0.4233
Epoch 18/600
11/11 [==============================] - 124s 11s/step - loss: 0.1563 - acc: 0.8088 - dice_axon: 0.0036 - dice_myelin: 0.4610 - val_loss: 0.2322 - val_acc: 0.7682 - val_dice_axon: 0.0024 - val_dice_myelin: 0.5184
Epoch 19/600
11/11 [==============================] - 123s 11s/step - loss: 0.1384 - acc: 0.8266 - dice_axon: 0.0032 - dice_myelin: 0.4926 - val_loss: 0.1157 - val_acc: 0.8848 - val_dice_axon: 0.0010 - val_dice_myelin: 0.6530
Epoch 20/600
11/11 [==============================] - 132s 12s/step - loss: 0.1661 - acc: 0.8005 - dice_axon: 0.0024 - dice_myelin: 0.5048 - val_loss: 0.1930 - val_acc: 0.8073 - val_dice_axon: 0.0013 - val_dice_myelin: 0.4882
Epoch 21/600
11/11 [==============================] - 117s 11s/step - loss: 0.1474 - acc: 0.8257 - dice_axon:

Epoch 53/600
11/11 [==============================] - 131s 12s/step - loss: 0.1605 - acc: 0.8435 - dice_axon: 0.0010 - dice_myelin: 0.6029 - val_loss: 0.2032 - val_acc: 0.7968 - val_dice_axon: 2.8035e-04 - val_dice_myelin: 0.4509
Epoch 54/600
11/11 [==============================] - 116s 11s/step - loss: 0.1386 - acc: 0.8476 - dice_axon: 9.0610e-04 - dice_myelin: 0.5445 - val_loss: 0.2281 - val_acc: 0.7720 - val_dice_axon: 5.5151e-04 - val_dice_myelin: 0.5298
Epoch 55/600
11/11 [==============================] - 123s 11s/step - loss: 0.1508 - acc: 0.8372 - dice_axon: 7.3429e-04 - dice_myelin: 0.5346 - val_loss: 0.1242 - val_acc: 0.8760 - val_dice_axon: 4.5409e-04 - val_dice_myelin: 0.5361
Epoch 56/600
11/11 [==============================] - 124s 11s/step - loss: 0.1374 - acc: 0.8445 - dice_axon: 0.0010 - dice_myelin: 0.5736 - val_loss: 0.1883 - val_acc: 0.8117 - val_dice_axon: 1.6278e-04 - val_dice_myelin: 0.4311
Epoch 57/600
11/11 [==============================] - 132s 12s/step - lo

11/11 [==============================] - 123s 11s/step - loss: 0.1367 - acc: 0.8980 - dice_axon: 7.2015e-04 - dice_myelin: 0.6942 - val_loss: 0.1882 - val_acc: 0.8119 - val_dice_axon: 2.0637e-04 - val_dice_myelin: 0.5037
Epoch 89/600
11/11 [==============================] - 131s 12s/step - loss: 0.1536 - acc: 0.8795 - dice_axon: 5.0726e-04 - dice_myelin: 0.6801 - val_loss: 0.1920 - val_acc: 0.8081 - val_dice_axon: 1.2237e-04 - val_dice_myelin: 0.4496
Epoch 90/600
11/11 [==============================] - 123s 11s/step - loss: 0.1459 - acc: 0.8886 - dice_axon: 6.8539e-04 - dice_myelin: 0.6775 - val_loss: 0.2152 - val_acc: 0.7848 - val_dice_axon: 2.2001e-04 - val_dice_myelin: 0.5463
Epoch 91/600
11/11 [==============================] - 116s 11s/step - loss: 0.1457 - acc: 0.8673 - dice_axon: 5.2951e-04 - dice_myelin: 0.6619 - val_loss: 0.1142 - val_acc: 0.8859 - val_dice_axon: 1.2493e-04 - val_dice_myelin: 0.6386
Epoch 92/600
11/11 [==============================] - 131s 12s/step - loss: 0

Epoch 123/600
11/11 [==============================] - 130s 12s/step - loss: 0.1395 - acc: 0.8802 - dice_axon: 6.8113e-04 - dice_myelin: 0.6725 - val_loss: 0.1113 - val_acc: 0.8887 - val_dice_axon: 2.8465e-04 - val_dice_myelin: 0.6615
Epoch 124/600
11/11 [==============================] - 116s 11s/step - loss: 0.1274 - acc: 0.8826 - dice_axon: 5.7781e-04 - dice_myelin: 0.6696 - val_loss: 0.1911 - val_acc: 0.8090 - val_dice_axon: 1.7966e-04 - val_dice_myelin: 0.5044
Epoch 125/600
11/11 [==============================] - 123s 11s/step - loss: 0.1485 - acc: 0.8694 - dice_axon: 5.5057e-04 - dice_myelin: 0.6042 - val_loss: 0.1960 - val_acc: 0.8041 - val_dice_axon: 1.9780e-04 - val_dice_myelin: 0.4603
Epoch 126/600
11/11 [==============================] - 130s 12s/step - loss: 0.1417 - acc: 0.8782 - dice_axon: 4.9234e-04 - dice_myelin: 0.6872 - val_loss: 0.2178 - val_acc: 0.7823 - val_dice_axon: 1.6245e-04 - val_dice_myelin: 0.5416
Epoch 127/600
11/11 [==============================] - 116s 

...

11/11 [==============================] - 123s 11s/step - loss: 0.0682 - acc: 0.9677 - dice_axon: 0.9425 - dice_myelin: 0.8115 - val_loss: 0.1979 - val_acc: 0.8021 - val_dice_axon: 0.4358 - val_dice_myelin: 0.4685
Epoch 446/600
11/11 [==============================] - 122s 11s/step - loss: 0.0578 - acc: 0.9737 - dice_axon: 0.9440 - dice_myelin: 0.8429 - val_loss: 0.2048 - val_acc: 0.7952 - val_dice_axon: 0.5271 - val_dice_myelin: 0.5357
Epoch 447/600
11/11 [==============================] - 122s 11s/step - loss: 0.0664 - acc: 0.9708 - dice_axon: 0.9398 - dice_myelin: 0.8349 - val_loss: 0.0805 - val_acc: 0.9195 - val_dice_axon: 0.6780 - val_dice_myelin: 0.6727
Epoch 448/600
11/11 [==============================] - 130s 12s/step - loss: 0.0733 - acc: 0.9622 - dice_axon: 0.9437 - dice_myelin: 0.8119 - val_loss: 0.1815 - val_acc: 0.8185 - val_dice_axon: 0.4904 - val_dice_myelin: 0.5212
Epoch 449/600
11/11 [==============================] - 116s 11s/step - loss: 0.0575 - acc: 0.9662 - dice_a

11/11 [==============================] - 123s 11s/step - loss: 0.0623 - acc: 0.9689 - dice_axon: 0.9443 - dice_myelin: 0.8337 - val_loss: 0.2025 - val_acc: 0.7975 - val_dice_axon: 0.4254 - val_dice_myelin: 0.4650
Epoch 482/600
11/11 [==============================] - 122s 11s/step - loss: 0.0747 - acc: 0.9671 - dice_axon: 0.9428 - dice_myelin: 0.8094 - val_loss: 0.2028 - val_acc: 0.7972 - val_dice_axon: 0.5269 - val_dice_myelin: 0.5347
Epoch 483/600
11/11 [==============================] - 122s 11s/step - loss: 0.0608 - acc: 0.9685 - dice_axon: 0.9419 - dice_myelin: 0.8101 - val_loss: 0.0813 - val_acc: 0.9187 - val_dice_axon: 0.6752 - val_dice_myelin: 0.6709
Epoch 484/600
11/11 [==============================] - 122s 11s/step - loss: 0.0705 - acc: 0.9634 - dice_axon: 0.9388 - dice_myelin: 0.7998 - val_loss: 0.1781 - val_acc: 0.8219 - val_dice_axon: 0.4930 - val_dice_myelin: 0.5068
Epoch 485/600
11/11 [==============================] - 123s 11s/step - loss: 0.0706 - acc: 0.9583 - dice_a

11/11 [==============================] - 124s 11s/step - loss: 0.0575 - acc: 0.9684 - dice_axon: 0.9447 - dice_myelin: 0.8145 - val_loss: 0.1971 - val_acc: 0.8029 - val_dice_axon: 0.4460 - val_dice_myelin: 0.4727
Epoch 518/600
11/11 [==============================] - 123s 11s/step - loss: 0.0702 - acc: 0.9597 - dice_axon: 0.9470 - dice_myelin: 0.7778 - val_loss: 0.2034 - val_acc: 0.7966 - val_dice_axon: 0.5261 - val_dice_myelin: 0.5353
Epoch 519/600
11/11 [==============================] - 124s 11s/step - loss: 0.0570 - acc: 0.9730 - dice_axon: 0.9452 - dice_myelin: 0.8339 - val_loss: 0.0810 - val_acc: 0.9190 - val_dice_axon: 0.6752 - val_dice_myelin: 0.6659
Epoch 520/600
11/11 [==============================] - 123s 11s/step - loss: 0.0702 - acc: 0.9709 - dice_axon: 0.9431 - dice_myelin: 0.8385 - val_loss: 0.1778 - val_acc: 0.8222 - val_dice_axon: 0.4973 - val_dice_myelin: 0.5246
Epoch 521/600
11/11 [==============================] - 131s 12s/step - loss: 0.0671 - acc: 0.9720 - dice_a

11/11 [==============================] - 124s 11s/step - loss: 0.0695 - acc: 0.9677 - dice_axon: 0.9441 - dice_myelin: 0.8035 - val_loss: 0.1929 - val_acc: 0.8071 - val_dice_axon: 0.4429 - val_dice_myelin: 0.4570
Epoch 554/600
11/11 [==============================] - 123s 11s/step - loss: 0.0698 - acc: 0.9669 - dice_axon: 0.9356 - dice_myelin: 0.8206 - val_loss: 0.2088 - val_acc: 0.7912 - val_dice_axon: 0.5254 - val_dice_myelin: 0.5399
Epoch 555/600
11/11 [==============================] - 123s 11s/step - loss: 0.0824 - acc: 0.9701 - dice_axon: 0.9444 - dice_myelin: 0.8096 - val_loss: 0.0798 - val_acc: 0.9202 - val_dice_axon: 0.6746 - val_dice_myelin: 0.6828
Epoch 556/600
11/11 [==============================] - 124s 11s/step - loss: 0.0684 - acc: 0.9638 - dice_axon: 0.9392 - dice_myelin: 0.7928 - val_loss: 0.1798 - val_acc: 0.8202 - val_dice_axon: 0.4902 - val_dice_myelin: 0.5155
Epoch 557/600
11/11 [==============================] - 123s 11s/step - loss: 0.0587 - acc: 0.9604 - dice_a

11/11 [==============================] - 124s 11s/step - loss: 0.0715 - acc: 0.9648 - dice_axon: 0.9485 - dice_myelin: 0.7950 - val_loss: 0.1977 - val_acc: 0.8023 - val_dice_axon: 0.4463 - val_dice_myelin: 0.4754
Epoch 590/600
11/11 [==============================] - 123s 11s/step - loss: 0.0597 - acc: 0.9620 - dice_axon: 0.9469 - dice_myelin: 0.7898 - val_loss: 0.2048 - val_acc: 0.7952 - val_dice_axon: 0.5256 - val_dice_myelin: 0.5411
Epoch 591/600
11/11 [==============================] - 125s 11s/step - loss: 0.0779 - acc: 0.9618 - dice_axon: 0.9452 - dice_myelin: 0.7693 - val_loss: 0.0808 - val_acc: 0.9192 - val_dice_axon: 0.6834 - val_dice_myelin: 0.6761
Epoch 592/600
11/11 [==============================] - 126s 11s/step - loss: 0.0581 - acc: 0.9635 - dice_axon: 0.9491 - dice_myelin: 0.8218 - val_loss: 0.1803 - val_acc: 0.8197 - val_dice_axon: 0.4922 - val_dice_myelin: 0.5184
Epoch 593/600
11/11 [==============================] - 131s 12s/step - loss: 0.0691 - acc: 0.9665 - dice_a

#### 1.5. Monitor the training with Tensorboard

[TensorBoard](https://www.tensorflow.org/guide/summaries_and_tensorboard) can be used to monitor the training procedure (loss and accuracy graphs, gradients, activations, identify bugs, etc.). To run TensorBoard, activate ADS virtual environment and run:
```
tensorboard --logdir PATH_MODEL --port 6006
```
where `PATH_MODEL` corresponds to this notebook's `path_model` variable (folder where model is being trained), and `port` is the port number where the TensorBoard local web server will be sent to (e.g., port 6006). Once the command is run, open a web browser with the address:
```
http://localhost:6006/
```

#### 1.6. Resume training from checkpoint

To resume training from a checkpoint, change the "checkpoint" parameter in "config" from None to "loss" or "accuracy".

In [ ]:
path_model = "../models/Path_of_the_model" # Path of the model where the checkpoint is saved

train_model(str(path_training), str(path_model), config)

### 2. Test the trained model
#### 2.1. Set the path of the test image to be segmented with the trained model

In [ ]:
# Modify the lines below to use your image
path_img = Path("../AxonDeepSeg/data_test")
file_img = "image.png"

#### 2.2. Launch the image segmentation

In [ ]:
# In case you want to test the segmentation with a pre-trained model created using this notebook,
# uncomment the line below.
#path_model = Path("../AxonDeepSeg/models/default_SEM_model")

# reset the tensorflow graph for new testing
tf.reset_default_graph()
prediction = axon_segmentation(path_img, file_img, path_model, config_network,  resampled_resolutions=0.1, verbosity_level=3)

#### 2.3. Display the resulted segmentation

In [ ]:
file_img_seg = 'AxonDeepSeg.png'  # axon+myelin segmentation

img_seg = ads.imread(path_img / file_img_seg)
img = ads.imread(path_img / file_img)
# Note: The arguments of the two function calls above use the pathlib syntax for path concatenation.

fig, axes = plt.subplots(1,2, figsize=(13,10))
ax1, ax2 = axes[0], axes[1]
ax1.set_title('Original image')
ax1.imshow(img, cmap='gray')
ax2.set_title('Prediction with the trained model')
ax2.imshow(img_seg,cmap='gray')
plt.show()